# CIS580 Final Homework

## 1. Two View Stereo

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import cv2

EPS = 1e-8

from utils import viz_camera_poses

### 1.0 Load Data

Note: you can uncomment the `viz_camera_poses([view_i, view_j])` to interactively understand the camera configuration, you will get some coordinates colored R-X, G-Y, B-Z. Press `i` in the interactive pyrender window to show the world coordinate.

You will find that in the world frame, for each camera, X+ direction is vertically upward, the Y+ direction towards the right, and the Z+ forward.

You will get views that are lying horizontally. In each image, the left-up corner has integer pixel coordinate [u,v] = [0,0]. The first index/row index of the image corresponds to Y or v, and the second index/col index of the image corresponds to X or u.

In [ ]:
# Load data
from dataloader import load_middlebury_data
DATA = load_middlebury_data("data/templeRing")
# reference: https://vision.middlebury.edu/mview/

view_i, view_j = DATA[0], DATA[3]
# viz_camera_poses([view_i, view_j])

plt.subplot(1, 2, 1)
plt.title("Sample: left view (reference)")
plt.imshow(view_i["rgb"])
plt.subplot(1, 2, 2)
plt.title("Sample: right view")
plt.imshow(view_j["rgb"])
plt.show()

### 1.1 Rectify two views

<!-- Complete the function `compute_right2left_transformation` and `compute_rectification_R`; we have implemented the calibration `rectify_2view`  for you after you get the `R_irect` -->

In [ ]:
from two_view_stereo import (
    rectify_2view,
    compute_rectification_R,
    compute_right2left_transformation,
)

R_wi, T_wi = view_i["R"], view_i["T"][:, None]  # p_i = R_wi @ p_w + T_wi
R_wj, T_wj = view_j["R"], view_j["T"][:, None]  # p_j = R_wj @ p_w + T_wj

R_ji, T_ji, B = compute_right2left_transformation(R_wi, T_wi, R_wj, T_wj)
assert T_ji[1, 0] > 0, "here we assume view i should be on the left, not on the right"

R_irect = compute_rectification_R(T_ji)

rgb_i_rect, rgb_j_rect, K_i_corr, K_j_corr = rectify_2view(
    view_i["rgb"],
    view_j["rgb"],
    R_irect,
    R_irect @ R_ji,
    view_i["K"],
    view_j["K"],
    u_padding=20,
    v_padding=20,
)


In [ ]:
plt.subplot(2, 2, 1)
plt.title("input view i")
plt.imshow(cv2.rotate(view_i["rgb"], cv2.ROTATE_90_COUNTERCLOCKWISE))
plt.subplot(2, 2, 2)
plt.title("input view j")
plt.imshow(cv2.rotate(view_j["rgb"], cv2.ROTATE_90_COUNTERCLOCKWISE))
plt.subplot(2, 2, 3)
plt.title("rect view i")
plt.imshow(cv2.rotate(rgb_i_rect, cv2.ROTATE_90_COUNTERCLOCKWISE))
plt.subplot(2, 2, 4)
plt.title("rect view j")
plt.imshow(cv2.rotate(rgb_j_rect, cv2.ROTATE_90_COUNTERCLOCKWISE))
plt.tight_layout()
plt.show()

### 1.2 Compute the disparity

In [ ]:
from two_view_stereo import image2patch, ssd_kernel

assert K_i_corr[1, 1] == K_j_corr[1, 1], "This hw assumes the same focal Y length"
assert (K_i_corr[0] == K_j_corr[0]).all(), "This hw assumes the same K on X dim"
assert (
    rgb_i_rect.shape == rgb_j_rect.shape
), "This hw makes rectified two views to have the same shape"

h, w = rgb_i_rect.shape[:2]

d0 = K_j_corr[1, 2] - K_i_corr[1, 2]

patches_i = image2patch(rgb_i_rect.astype(float) / 255.0, 3)  # [h,w,k*k,3]
patches_j = image2patch(rgb_j_rect.astype(float) / 255.0, 3)  # [h,w,k*k,3]

vi_idx, vj_idx = np.arange(h), np.arange(h)
disp_candidates = vi_idx[:, None] - vj_idx[None, :] + d0
valid_disp_mask = disp_candidates > 0.0

# as an example
u = 400

buf_i, buf_j = patches_i[:, u], patches_j[:, u]
value = ssd_kernel(buf_i, buf_j)  # each row is one pix from left, col is the disparity
# Students, why we compute this `_upper` ?
_upper = value.max() + 1.0
value[~valid_disp_mask] = _upper


In [ ]:
plt.subplot(1,2,1)
# Viz the  disparity-cost of u=500, v=200 on left view
v = 200
plt.title("Cost-Disparity of one left pixel")
plt.xlabel("Disparity")
plt.ylabel("Cost")
plt.plot(disp_candidates[v], value[v])
plt.subplot(1,2,2)
plt.title("The cost map of one left horizon col")
plt.xlabel("Disparity")
plt.ylabel("left pixel coordinates  v_L")
plt.imshow(value)
plt.tight_layout()
plt.show()

Now let's do the Left-Right consistency: in above right figure, the vertical axis is the left pixel coordinates and the horizontal axis is the right pixel (disparity). 

When we find the minimal cost right view pixel for our reference pixel (e.g. find the minimum of the left figure), we only check the best matched right pixel patch, which corresponds to finding the minimum along the horizontal axis of the right cost map. 

However, the matching from the left to the right and from the right to the left should be consistent.
So we should also check the minimum of the vertical axis and see whether this two minimum agree with each other, below is an example

In [ ]:
# example for the pixel (u=500,v=300) from the left view
v = 300
best_matched_right_pixel = value[v].argmin()
best_matched_left_pixel = value[:,best_matched_right_pixel].argmin()
print(v, best_matched_left_pixel)
consistent_flag = best_matched_left_pixel == v
print(consistent_flag)

# example for the pixel (u=500,v=380) from the left view
v = 380
best_matched_right_pixel = value[v].argmin()
best_matched_left_pixel = value[:,best_matched_right_pixel].argmin()
print(v, best_matched_left_pixel)
consistent_flag = best_matched_left_pixel == v
print(consistent_flag)


Based on above example, iterate over all u, complete the function `compute_disparity_map`.

Tips: for vectorized operation, you may use `take_along_axis` with `argmin` from numpy, please check the documentation from numpy to understand how to use the function

In [ ]:
from two_view_stereo import compute_disparity_map

disp_map, consistency_mask = compute_disparity_map(
    rgb_i_rect, rgb_j_rect, d0=K_j_corr[1, 2] - K_i_corr[1, 2], k_size=5
)


In [ ]:
plt.imshow(cv2.rotate(consistency_mask, cv2.ROTATE_90_COUNTERCLOCKWISE))
plt.title("L-R consistency check mask")
plt.show()

### 1.3 Compute the depth map and point cloud reconstruction

In [ ]:
from two_view_stereo import compute_dep_and_pcl

# * 3. compute depth map and filter them
dep_map, xyz_cam = compute_dep_and_pcl(disp_map, B, K_i_corr)

plt.subplot(1, 3, 1)
plt.title("rgb")
plt.imshow(cv2.rotate(rgb_i_rect, cv2.ROTATE_90_COUNTERCLOCKWISE))
plt.subplot(1, 3, 2)
plt.title("raw disparity")
plt.imshow(cv2.rotate(disp_map, cv2.ROTATE_90_COUNTERCLOCKWISE))
plt.colorbar()
plt.subplot(1, 3, 3)
plt.title("raw depth")
plt.imshow(cv2.rotate(dep_map, cv2.ROTATE_90_COUNTERCLOCKWISE))
plt.colorbar()
plt.tight_layout()
plt.show()


### 1.4 Postprocessing and visualization

In [ ]:
from two_view_stereo import postprocess

mask, pcl_world, pcl_cam, pcl_color = postprocess(
    dep_map,
    rgb_i_rect,
    xyz_cam,
    R_wc=R_irect @ R_wi,
    T_wc=R_irect @ T_wi,
    consistency_mask=consistency_mask,
    z_near=0.5,
    z_far=0.6,
)

mask = (mask > 0).astype(np.float)

plt.subplot(1, 3, 1)
plt.title("rgb")
plt.imshow(cv2.rotate(rgb_i_rect, cv2.ROTATE_90_COUNTERCLOCKWISE))
plt.subplot(1, 3, 2)
plt.title("postproc disparity")
plt.imshow(cv2.rotate(disp_map * mask, cv2.ROTATE_90_COUNTERCLOCKWISE))
plt.colorbar()
plt.subplot(1, 3, 3)
plt.title("postproc depth")
plt.imshow(cv2.rotate(dep_map * mask, cv2.ROTATE_90_COUNTERCLOCKWISE))
plt.colorbar()
plt.show()


In [ ]:
import k3d
import numpy as np

def viz_3d_embedded(pcl, color):
    plot = k3d.plot(camera_auto_fit=True)
    color = color.astype(np.uint8)
    color32 = (color[:, 0] * 256**2 + color[:, 1] * 256**1 + color[:, 2] * 256**0).astype(
        np.uint32
    )
    plot += k3d.points(pcl.astype(float), color32, point_size=0.001, shader="flat")
    plot.display()

#### SSD Two view reconstruction results

In [ ]:
viz_3d_embedded(pcl_world, pcl_color.astype(np.uint8))

#### SAD Two view reconstruction results
<!-- ### 1.5 Put things together and try different Similarity -->

In [ ]:
from two_view_stereo import two_view ,sad_kernel

pcl_sad, pcl_color_sad, disp_map_sad, dep_map_sad = two_view(DATA[0], DATA[2], 5, sad_kernel)
viz_3d_embedded(pcl_sad, pcl_color_sad.astype(np.uint8))

#### ZNCC Two view reconstruction results

In [ ]:
from two_view_stereo import two_view, zncc_kernel

pcl_zncc, pcl_color_zncc, disp_map_zncc, dep_map_zncc = two_view(DATA[0], DATA[2], 5, zncc_kernel)
viz_3d_embedded(pcl_zncc, pcl_color_zncc.astype(np.uint8))


### 1.5 Aggregation

In [ ]:
from two_view_stereo import two_view, ssd_kernel, sad_kernel, zncc_kernel

pcl_list, pcl_color_list, disp_map_list, dep_map_list = [], [], [], []
pairs = [(0, 2), (2, 4), (5, 7), (8, 10), (13, 15), (16, 18), (19, 21), (22, 24), (25, 27)]
# for i in range(13, 28, 3):
for pair in pairs:
    i,j = pair
    _pcl, _pcl_color, _disp_map, _dep_map = two_view(DATA[i], DATA[j], 5, sad_kernel)
    pcl_list.append(_pcl)
    pcl_color_list.append(_pcl_color)
    disp_map_list.append(_disp_map)
    dep_map_list.append(_dep_map)

In [ ]:
plot = k3d.plot(camera_auto_fit=True)
for pcl, color in zip(pcl_list, pcl_color_list):
    color = color.astype(np.uint8)
    color32 = (color[:, 0] * 256**2 + color[:, 1] * 256**1 + color[:, 2] * 256**0).astype(
        np.uint32
    )
    plot += k3d.points(pcl.astype(float), color32, point_size=0.001, shader="flat")
plot.display()